**ETL Creacion Staging**

***1. Creación stg_baq_compras***

In [1]:
#pip install mysql-connector-python

In [2]:
import requests
import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError

In [3]:
# URL del archivo en GitHub
url = 'https://github.com/demonory/PAAD_G17/raw/main/Fuentes/BAQ_compras.csv'

# Descargar el archivo
response = requests.get(url)
with open('BAQ_compras.csv', 'wb') as file:
    file.write(response.content)

# Leer el archivo CSV
df = pd.read_csv('BAQ_compras.csv')
df.head()


,warehouse_code,region_code,id,delivery_date,product_id,sku,name,supplier_id,price,quantity
0,BAQ,BAQ,8475266,9/8/2022,660,BAQ-FRU1-CAT1-111:277:659:660,Granadilla Estándar - Kg,61,5950.0,5.0
1,BAQ,BAQ,8554498,9/12/2022,660,BAQ-FRU1-CAT1-111:277:659:660,Granadilla Estándar - Kg,61,5950.0,8.0
2,BAQ,BAQ,8682054,9/19/2022,660,BAQ-FRU1-CAT1-111:277:659:660,Granadilla Estándar - Kg,61,7500.0,2.0
3,BAQ,BAQ,8715673,9/22/2022,660,BAQ-FRU1-CAT1-111:277:659:660,Granadilla Estándar - Kg,19,9000.0,6.0
4,BAQ,BAQ,8842178,9/29/2022,660,BAQ-FRU1-CAT1-111:277:659:660,Granadilla Estándar - Kg,61,7200.0,3.0


In [4]:
# Configuración de la conexión a la base de datos
user = 'dmonroy3'
password = 'SamuelAlejo2020%'
host = 'localhost'
database = 'paad_g17'
schema = 'paad_g17'
table_name = 'stg_baq_compras'

# Crear la conexión
engine = create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}/{database}')

# Verificar y crear el esquema si no existe
try:
    with engine.connect() as connection:
        # Verificar si el esquema existe
        result = connection.execute(text(f"SELECT SCHEMA_NAME FROM information_schema.SCHEMATA WHERE SCHEMA_NAME = '{schema}'"))
        schema_exists = result.fetchone()

        if not schema_exists:
            # Si el esquema no existe, crearlo
            connection.execute(text(f"CREATE SCHEMA {schema}"))
            print(f"Esquema '{schema}' creado correctamente.")
        else:
            print(f"Esquema '{schema}' ya existe.")

except SQLAlchemyError as e:
    print(f"Error al conectar a la base de datos o crear el esquema: {e}")

Esquema 'paad_g17' ya existe.


In [5]:
# Borrar la tabla si existe
try:
    with engine.connect() as connection:
        connection.execute(text(f"DROP TABLE IF EXISTS {table_name}"))
    print("Tabla borrada correctamente, si existía.")
except SQLAlchemyError as e:
    print(f"Error al borrar la tabla: {e}")

Tabla borrada correctamente, si existía.


In [6]:
def map_dtype_to_mysql(dtype):
    return 'VARCHAR(255)'

In [7]:
# Obtener los tipos de datos adecuados para cada columna del DataFrame
column_types = {column: map_dtype_to_mysql(dtype) for column, dtype in df.dtypes.items()}

# Crear la consulta CREATE TABLE utilizando los tipos de datos adecuados
create_table_query = f"CREATE TABLE {table_name} ("
for column, column_type in column_types.items():
    create_table_query += f"\n{column} {column_type},"
create_table_query = create_table_query.rstrip(',') + "\n)"

# Ejecutar la consulta para crear la tabla
try:
    with engine.connect() as connection:
        connection.execute(text(create_table_query))
    print("Tabla creada correctamente.")
except SQLAlchemyError as e:
    print(f"Error al crear la tabla: {e}")

Tabla creada correctamente.


In [8]:
# Ahora, insertar los datos en la tabla
try:
    num_rows_inserted = df.shape[0]  # Obtener el número de filas en el DataFrame
    df.to_sql(table_name, con=engine, if_exists='append', index=False)
    print(f"{num_rows_inserted} datos insertados correctamente.")
except SQLAlchemyError as e:
    print(f"Error al insertar datos en la tabla: {e}")

5103 datos insertados correctamente.


***2. Creación stg_products_baq***

In [9]:
# URL del archivo en GitHub
url = 'https://github.com/demonory/PAAD_G17/raw/main/Fuentes/Products_BAQ.csv'

# Descargar el archivo
response = requests.get(url)
with open('Products_BAQ.csv', 'wb') as file:
    file.write(response.content)

# Leer el archivo CSV
df = pd.read_csv('Products_BAQ.csv')
df.head()

,product_id,sku,name,category,region_code,product_category_id,mean_shelf_life,promised_lead_time,purchasing_unit,buy_unit,weight_parameter_apricot
0,660,BAQ-FRU1-CAT1-111:277:659:660,Granadilla Estándar - Kg,Frutas,BAQ,1,2,NaN,1.0,KG,1.0
1,676,BAQ-FRU1-CAT1-123:280:665:676,Kiwi Estándar - Kg,Frutas,BAQ,1,2,NaN,1.0,KG,1.0
2,116,BAQ-FRU1-CAT1-14:49:115:116,Limón Pajarito Estándar - Kg - 🤑 (Insuperable),Frutas,BAQ,1,2,NaN,1.0,KG,1.0
3,652,BAQ-FRU1-CAT1-157:273:651:652,Tomate de Árbol Maduración Mixta Estándar - De...,Verduras,BAQ,1,2,NaN,0.1,KG,1.0
4,118,BAQ-FRU1-CAT1-15:50:117:118,Limón Tahití Estándar - Kg,Frutas,BAQ,1,3,NaN,1.0,KG,1.0


In [10]:
# Configuración de la conexión a la base de datos
user = 'dmonroy3'
password = 'SamuelAlejo2020%'
host = 'localhost'
database = 'paad_g17'
schema = 'paad_g17'
table_name = 'stg_products_baq'

# Crear la conexión
engine = create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}/{database}')

# Verificar y crear el esquema si no existe
try:
    with engine.connect() as connection:
        # Verificar si el esquema existe
        result = connection.execute(text(f"SELECT SCHEMA_NAME FROM information_schema.SCHEMATA WHERE SCHEMA_NAME = '{schema}'"))
        schema_exists = result.fetchone()

        if not schema_exists:
            # Si el esquema no existe, crearlo
            connection.execute(text(f"CREATE SCHEMA {schema}"))
            print(f"Esquema '{schema}' creado correctamente.")
        else:
            print(f"Esquema '{schema}' ya existe.")

except SQLAlchemyError as e:
    print(f"Error al conectar a la base de datos o crear el esquema: {e}")

Esquema 'paad_g17' ya existe.


In [11]:
# Borrar la tabla si existe
try:
    with engine.connect() as connection:
        connection.execute(text(f"DROP TABLE IF EXISTS {table_name}"))
    print("Tabla borrada correctamente, si existía.")
except SQLAlchemyError as e:
    print(f"Error al borrar la tabla: {e}")

Tabla borrada correctamente, si existía.


In [12]:
def map_dtype_to_mysql(dtype):
    return 'VARCHAR(255)'

In [13]:
# Obtener los tipos de datos adecuados para cada columna del DataFrame
column_types = {column: map_dtype_to_mysql(dtype) for column, dtype in df.dtypes.items()}

# Crear la consulta CREATE TABLE utilizando los tipos de datos adecuados
create_table_query = f"CREATE TABLE {table_name} ("
for column, column_type in column_types.items():
    create_table_query += f"\n{column} {column_type},"
create_table_query = create_table_query.rstrip(',') + "\n)"

# Ejecutar la consulta para crear la tabla
try:
    with engine.connect() as connection:
        connection.execute(text(create_table_query))
    print("Tabla creada correctamente.")
except SQLAlchemyError as e:
    print(f"Error al crear la tabla: {e}")

Tabla creada correctamente.


In [14]:
# Ahora, insertar los datos en la tabla
try:
    num_rows_inserted = df.shape[0]  # Obtener el número de filas en el DataFrame
    df.to_sql(table_name, con=engine, if_exists='append', index=False)
    print(f"{num_rows_inserted} datos insertados correctamente.")
except SQLAlchemyError as e:
    print(f"Error al insertar datos en la tabla: {e}")

137 datos insertados correctamente.


***3. Creación stg_waste_percentage_by_age***

In [15]:
# URL del archivo en GitHub
url = 'https://github.com/demonory/PAAD_G17/raw/main/Fuentes/waste_percentage_by_age.csv'

# Descargar el archivo
response = requests.get(url)
with open('waste_percentage_by_age.csv', 'wb') as file:
    file.write(response.content)

# Leer el archivo CSV
df = pd.read_csv('waste_percentage_by_age.csv')
# Cambiar el nombre de las columnas
df.rename(columns={'waste_percentage (%)': 'waste_percentage', 'shelf life': 'shelf_life'}, inplace=True)
df.head()

,shelf_life,age,waste_percentage
0,1,1,50.28
1,1,2,75.27
2,1,3,87.71
3,1,4,93.89
4,1,5,96.96


In [16]:
# Configuración de la conexión a la base de datos
user = 'dmonroy3'
password = 'SamuelAlejo2020%'
host = 'localhost'
database = 'paad_g17'
schema = 'paad_g17'
table_name = 'stg_waste_percentage_by_age'

# Crear la conexión
engine = create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}/{database}')

# Verificar y crear el esquema si no existe
try:
    with engine.connect() as connection:
        # Verificar si el esquema existe
        result = connection.execute(text(f"SELECT SCHEMA_NAME FROM information_schema.SCHEMATA WHERE SCHEMA_NAME = '{schema}'"))
        schema_exists = result.fetchone()

        if not schema_exists:
            # Si el esquema no existe, crearlo
            connection.execute(text(f"CREATE SCHEMA {schema}"))
            print(f"Esquema '{schema}' creado correctamente.")
        else:
            print(f"Esquema '{schema}' ya existe.")

except SQLAlchemyError as e:
    print(f"Error al conectar a la base de datos o crear el esquema: {e}")

Esquema 'paad_g17' ya existe.


In [17]:
# Borrar la tabla si existe
try:
    with engine.connect() as connection:
        connection.execute(text(f"DROP TABLE IF EXISTS {table_name}"))
    print("Tabla borrada correctamente, si existía.")
except SQLAlchemyError as e:
    print(f"Error al borrar la tabla: {e}")


Tabla borrada correctamente, si existía.


In [18]:
def map_dtype_to_mysql(dtype):
    return 'VARCHAR(255)'

In [19]:
# Obtener los tipos de datos adecuados para cada columna del DataFrame
column_types = {column: map_dtype_to_mysql(dtype) for column, dtype in df.dtypes.items()}

# Crear la consulta CREATE TABLE utilizando los tipos de datos adecuados
create_table_query = f"CREATE TABLE {table_name} ("
for column, column_type in column_types.items():
    create_table_query += f"\n{column} {column_type},"
create_table_query = create_table_query.rstrip(',') + "\n)"

# Ejecutar la consulta para crear la tabla
try:
    with engine.connect() as connection:
        connection.execute(text(create_table_query))
    print("Tabla creada correctamente.")
except SQLAlchemyError as e:
    print(f"Error al crear la tabla: {e}")

Tabla creada correctamente.


In [20]:
# Ahora, insertar los datos en la tabla
try:
    num_rows_inserted = df.shape[0]  # Obtener el número de filas en el DataFrame
    df.to_sql(table_name, con=engine, if_exists='append', index=False)
    print(f"{num_rows_inserted} datos insertados correctamente.")
except SQLAlchemyError as e:
    print(f"Error al insertar datos en la tabla: {e}")

7921 datos insertados correctamente.


***4. Creación stg_revision_precios_sipsa***

In [21]:
# URL del archivo en GitHub
url = 'https://github.com/demonory/PAAD_G17/raw/main/Fuentes/Revision_precios_sipsa.csv'

# Descargar el archivo
response = requests.get(url)
with open('Revision_precios_sipsa.csv', 'wb') as file:
    file.write(response.content)

# Leer el archivo CSV
df = pd.read_csv('Revision_precios_sipsa.csv')
# Renombrar columnas
df.rename(columns={'Unidad de compra': 'unidad_de_compra', 
                   'Precio por kilo': 'precio_por_kilo', 
                   'AVERAGE de Precio promedio por kilogramo*': 'average_precio_promedio_por_kilogramo', 
                   'Surplus': 'surplus'}, inplace=True)
df.head()

,warehouse_code,region_code,id,delivery_date,AÃ±o,Mes,product_id,sku,name,supplier_id,price,quantity,unidad_de_compra,precio_por_kilo,Cadena2,Cadena1,average_precio_promedio_por_kilogramo,surplus
0,BAQ,BAQ,10475443,10/01/2023,2023,1,100,BAQ-FRU1-CAT1-2:42:99:100,Aguacate Maduro - Kg,383,10000,10,1.0,10000.0,2023-1-Aguacate Maduro - Kg,NaN,5496,81.95%
1,BAQ,BAQ,11445810,1/04/2023,2023,4,100,BAQ-FRU1-CAT1-2:42:99:100,Aguacate Maduro - Kg,383,7000,50,1.0,7000.0,2023-4-Aguacate Maduro - Kg,NaN,5572,25.63%
2,BAQ,BAQ,11460941,3/04/2023,2023,4,100,BAQ-FRU1-CAT1-2:42:99:100,Aguacate Maduro - Kg,383,7000,40,1.0,7000.0,2023-4-Aguacate Maduro - Kg,NaN,5572,25.63%
3,BAQ,BAQ,11508565,10/04/2023,2023,4,100,BAQ-FRU1-CAT1-2:42:99:100,Aguacate Maduro - Kg,383,7000,60,1.0,7000.0,2023-4-Aguacate Maduro - Kg,NaN,5572,25.63%
4,BAQ,BAQ,11514234,11/04/2023,2023,4,100,BAQ-FRU1-CAT1-2:42:99:100,Aguacate Maduro - Kg,382,7000,60,1.0,7000.0,2023-4-Aguacate Maduro - Kg,NaN,5572,25.63%


In [22]:
# Configuración de la conexión a la base de datos
user = 'dmonroy3'
password = 'SamuelAlejo2020%'
host = 'localhost'
database = 'paad_g17'
schema = 'paad_g17'
table_name = 'stg_revision_precios_sipsa'

# Crear la conexión
engine = create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}/{database}')

# Verificar y crear el esquema si no existe
try:
    with engine.connect() as connection:
        # Verificar si el esquema existe
        result = connection.execute(text(f"SELECT SCHEMA_NAME FROM information_schema.SCHEMATA WHERE SCHEMA_NAME = '{schema}'"))
        schema_exists = result.fetchone()

        if not schema_exists:
            # Si el esquema no existe, crearlo
            connection.execute(text(f"CREATE SCHEMA {schema}"))
            print(f"Esquema '{schema}' creado correctamente.")
        else:
            print(f"Esquema '{schema}' ya existe.")

except SQLAlchemyError as e:
    print(f"Error al conectar a la base de datos o crear el esquema: {e}")

Esquema 'paad_g17' ya existe.


In [23]:
# Borrar la tabla si existe
try:
    with engine.connect() as connection:
        connection.execute(text(f"DROP TABLE IF EXISTS {table_name}"))
    print("Tabla borrada correctamente, si existía.")
except SQLAlchemyError as e:
    print(f"Error al borrar la tabla: {e}")


Tabla borrada correctamente, si existía.


In [24]:
def map_dtype_to_mysql(dtype):
    return 'VARCHAR(255)'

In [25]:
# Obtener los tipos de datos adecuados para cada columna del DataFrame
column_types = {column: map_dtype_to_mysql(dtype) for column, dtype in df.dtypes.items()}

# Crear la consulta CREATE TABLE utilizando los tipos de datos adecuados
create_table_query = f"CREATE TABLE {table_name} ("
for column, column_type in column_types.items():
    create_table_query += f"\n{column} {column_type},"
create_table_query = create_table_query.rstrip(',') + "\n)"

# Ejecutar la consulta para crear la tabla
try:
    with engine.connect() as connection:
        connection.execute(text(create_table_query))
    print("Tabla creada correctamente.")
except SQLAlchemyError as e:
    print(f"Error al crear la tabla: {e}")

Tabla creada correctamente.


In [26]:
# Ahora, insertar los datos en la tabla
try:
    num_rows_inserted = df.shape[0]  # Obtener el número de filas en el DataFrame
    df.to_sql(table_name, con=engine, if_exists='append', index=False)
    print(f"{num_rows_inserted} datos insertados correctamente.")
except SQLAlchemyError as e:
    print(f"Error al insertar datos en la tabla: {e}")

5103 datos insertados correctamente.


***5. Creación stg_datos_con_clusters***

In [27]:
# URL del archivo en GitHub
url = 'https://github.com/demonory/PAAD_G17/raw/main/Fuentes/datos_con_clusters.csv'

# Descargar el archivo
response = requests.get(url)
with open('datos_con_clusters.csv', 'wb') as file:
    file.write(response.content)

# Leer el archivo CSV
df = pd.read_csv('datos_con_clusters.csv')
df = df.drop(df.columns[0], axis=1)
df.head()

,Proveedor,Pedidos_atendidos,Volumen_portafolio,Volumen_category_portafolio,Minimo_pedido,Maximo_pedido,Prom_variacion_sipsa,Cluster
0,2,330,23,2,1.0,100.0,-0.080710,0
1,3,2,2,1,15.0,20.0,0.000000,0
2,9,34,4,1,1.0,160.0,0.027052,0
3,10,30,17,1,1.0,159.0,-0.103380,0
4,18,98,9,1,5.0,742.0,0.122707,0


In [28]:
# Configuración de la conexión a la base de datos
user = 'dmonroy3'
password = 'SamuelAlejo2020%'
host = 'localhost'
database = 'paad_g17'
schema = 'paad_g17'
table_name = 'stg_datos_con_clusters'

# Crear la conexión
engine = create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}/{database}')

# Verificar y crear el esquema si no existe
try:
    with engine.connect() as connection:
        # Verificar si el esquema existe
        result = connection.execute(text(f"SELECT SCHEMA_NAME FROM information_schema.SCHEMATA WHERE SCHEMA_NAME = '{schema}'"))
        schema_exists = result.fetchone()

        if not schema_exists:
            # Si el esquema no existe, crearlo
            connection.execute(text(f"CREATE SCHEMA {schema}"))
            print(f"Esquema '{schema}' creado correctamente.")
        else:
            print(f"Esquema '{schema}' ya existe.")

except SQLAlchemyError as e:
    print(f"Error al conectar a la base de datos o crear el esquema: {e}")

Esquema 'paad_g17' ya existe.


In [29]:
# Borrar la tabla si existe
try:
    with engine.connect() as connection:
        connection.execute(text(f"DROP TABLE IF EXISTS {table_name}"))
    print("Tabla borrada correctamente, si existía.")
except SQLAlchemyError as e:
    print(f"Error al borrar la tabla: {e}")


Tabla borrada correctamente, si existía.


In [30]:
def map_dtype_to_mysql(dtype):
    return 'VARCHAR(255)'

In [31]:
# Obtener los tipos de datos adecuados para cada columna del DataFrame
column_types = {column: map_dtype_to_mysql(dtype) for column, dtype in df.dtypes.items()}

# Crear la consulta CREATE TABLE utilizando los tipos de datos adecuados
create_table_query = f"CREATE TABLE {table_name} ("
for column, column_type in column_types.items():
    create_table_query += f"\n{column} {column_type},"
create_table_query = create_table_query.rstrip(',') + "\n)"

# Ejecutar la consulta para crear la tabla
try:
    with engine.connect() as connection:
        connection.execute(text(create_table_query))
    print("Tabla creada correctamente.")
except SQLAlchemyError as e:
    print(f"Error al crear la tabla: {e}")

Tabla creada correctamente.


In [32]:
# Ahora, insertar los datos en la tabla
try:
    num_rows_inserted = df.shape[0]  # Obtener el número de filas en el DataFrame
    df.to_sql(table_name, con=engine, if_exists='append', index=False)
    print(f"{num_rows_inserted} datos insertados correctamente.")
except SQLAlchemyError as e:
    print(f"Error al insertar datos en la tabla: {e}")

141 datos insertados correctamente.


***6. Creación stg_descripcion_clusters_proveedores***

In [33]:
# URL del archivo en GitHub
url = 'https://github.com/demonory/PAAD_G17/raw/main/Fuentes/descripcion_clusters_proveedores.csv'

# Descargar el archivo
response = requests.get(url)
with open('descripcion_clusters_proveedores.csv', 'wb') as file:
    file.write(response.content)

# Leer el archivo CSV
df = pd.read_csv('descripcion_clusters_proveedores.csv', sep=';')
df.columns = df.columns.str.replace(' ', '_')
df.head()

,Cluster,Pedidos_atendidos,Volumen_portafolio,Volumen_category_portafolio,Minimo_pedido,Maximo_pedido,Prom_variacion_sipsa,Descripción
0,0,34.31,4.53,1.31,141.13,649.87,0.0627%,Proveedores de portafolio reducido - baja cap...
1,1,57.00,2.50,2.00,574.00,28500.00,0.2107%,Proveedores de portafolio reducido - amplia c...
2,2,88.87,21.20,2.38,83.50,11552.75,5.72%,Proveedores de amplio portafolio - amplia cap...
3,3,8.50,1.00,1.00,7700.00,16912.00,0.00%,Proveedores de portafolio reducido - amplia c...
4,4,1.20,1.00,1.00,744.56,744.56,0.00%,Proveedores de portafolio reducido - baja cap...


In [34]:
# Configuración de la conexión a la base de datos
user = 'dmonroy3'
password = 'SamuelAlejo2020%'
host = 'localhost'
database = 'paad_g17'
schema = 'paad_g17'
table_name = 'stg_descripcion_clusters_proveedores'

# Crear la conexión
engine = create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}/{database}')

# Verificar y crear el esquema si no existe
try:
    with engine.connect() as connection:
        # Verificar si el esquema existe
        result = connection.execute(text(f"SELECT SCHEMA_NAME FROM information_schema.SCHEMATA WHERE SCHEMA_NAME = '{schema}'"))
        schema_exists = result.fetchone()

        if not schema_exists:
            # Si el esquema no existe, crearlo
            connection.execute(text(f"CREATE SCHEMA {schema}"))
            print(f"Esquema '{schema}' creado correctamente.")
        else:
            print(f"Esquema '{schema}' ya existe.")

except SQLAlchemyError as e:
    print(f"Error al conectar a la base de datos o crear el esquema: {e}")

Esquema 'paad_g17' ya existe.


In [35]:
# Borrar la tabla si existe
try:
    with engine.connect() as connection:
        connection.execute(text(f"DROP TABLE IF EXISTS {table_name}"))
    print("Tabla borrada correctamente, si existía.")
except SQLAlchemyError as e:
    print(f"Error al borrar la tabla: {e}")


Tabla borrada correctamente, si existía.


In [36]:
def map_dtype_to_mysql(dtype):
    return 'VARCHAR(255)'

In [37]:
# Obtener los tipos de datos adecuados para cada columna del DataFrame
column_types = {column: map_dtype_to_mysql(dtype) for column, dtype in df.dtypes.items()}

# Crear la consulta CREATE TABLE utilizando los tipos de datos adecuados
create_table_query = f"CREATE TABLE {table_name} ("
for column, column_type in column_types.items():
    create_table_query += f"\n{column} {column_type},"
create_table_query = create_table_query.rstrip(',') + "\n)"

# Ejecutar la consulta para crear la tabla
try:
    with engine.connect() as connection:
        connection.execute(text(create_table_query))
    print("Tabla creada correctamente.")
except SQLAlchemyError as e:
    print(f"Error al crear la tabla: {e}")

Tabla creada correctamente.


In [38]:
# Ahora, insertar los datos en la tabla
try:
    num_rows_inserted = df.shape[0]  # Obtener el número de filas en el DataFrame
    df.to_sql(table_name, con=engine, if_exists='append', index=False)
    print(f"{num_rows_inserted} datos insertados correctamente.")
except SQLAlchemyError as e:
    print(f"Error al insertar datos en la tabla: {e}")

5 datos insertados correctamente.
